# Task 01 - Teoría

1. En un juego de suma cero para dos jugadores, ¿cómo funciona el algoritmo minimax para determinar la estrategia óptima para cada jugador? ¿Puede explicarnos el concepto de "valor minimax" y su importancia en este contexto?

2. Compare y contraste el algoritmo minimax con la poda alfa-beta. ¿Cómo mejora la poda alfa-beta la eficiencia del algoritmo minimax, particularmente en árboles de caza grandes? Proporcione un ejemplo para ilustrar la diferencia en la complejidad computacional entre la poda minimax y alfa-beta.

3. ¿Cuál es el papel de expectiminimax en juegos con incertidumbre, como aquellos que involucran nodos de azar o información oculta? ¿En qué se diferencia el expectiminimax del minimax en el manejo de resultados probabilísticos y cuáles son los desafíos clave que aborda?

# Task 02 - Connect Four

In [26]:
import numpy as np
import math
import random

In [27]:
# Constantes del juego

# Define el tamaño del tablero
CANT_FILAS = 6
CANT_COLUMNAS = 7

# Define el indice de los jugadores
JUGADOR = 0 
IA = 1

# Define las piezas de los jugadores
PIEZA_JUGADOR = 1
PIEZA_IA = 2

In [28]:
# Crea el tablero utilizando una matriz de 0s
def crearTablero():
    tablero = np.zeros((CANT_FILAS, CANT_COLUMNAS), dtype=int)
    return tablero

# Inserta la ficha en el tablero
def soltarFicha(tablero, fila, columna, pieza):
    tablero[fila][columna] = pieza

# Chequea si la columna es valida para soltar la ficha
def esValida(tablero, columna):
    return tablero[CANT_FILAS-1][columna] == 0

# Obtiene la siguiente fila vacia en la columna
def obtenerSiguienteFilaVacia(tablero, columna):
    for r in range(CANT_FILAS):
        if tablero[r][columna] == 0:
            return r

# Imprime el tablero
def imprimirTablero(tablero):
    tableroInvertido = np.flip(tablero, 0)
    filas, columnas = tableroInvertido.shape

    tableroInvertido = tableroInvertido.astype(int)

    for i in range(filas):
        print("|", end="")
        for j in range(columnas):
            print(f"{tableroInvertido[i,j]:2}", end=" |")
        print()
    
    print("-" * (columnas * 4))

    print("|", end=" ")
    for j in range(columnas):
        print(f"{j}", end=" | ")

# Chequea si alguien gano
def movimientoGanador(tablero, pieza):
    # Chequeo horizontal
    for i in range(CANT_COLUMNAS - 3):
        for j in range(CANT_FILAS):
            if tablero[j][i] == pieza and tablero[j][i+1] == pieza and tablero[j][i+2] == pieza and tablero[j][i+3] == pieza:
                return True
    
    # Chequeo vertical
    for i in range(CANT_COLUMNAS):
        for j in range(CANT_FILAS - 3):
            if tablero[j][i] == pieza and tablero[j+1][i] == pieza and tablero[j+2][i] == pieza and tablero[j+3][i] == pieza:
                return True
    
    # Chequeo diagonal positiva
    for i in range(CANT_COLUMNAS - 3):
        for j in range(CANT_FILAS - 3):
            if tablero[j][i] == pieza and tablero[j+1][i+1] == pieza and tablero[j+2][i+2] == pieza and tablero[j+3][i+3] == pieza:
                return True
    
    # Chequeo diagonal negativa
    for i in range(CANT_COLUMNAS - 3):
        for j in range(3, CANT_FILAS):
            if tablero[j][i] == pieza and tablero[j-1][i+1] == pieza and tablero[j-2][i+2] == pieza and tablero[j-3][i+3] == pieza:
                return True

    return False

# Obtiene una lista con las posiciones validas para soltar la ficha
def obtenerPosicionesValidas(tablero):
    posicionesValidas = []
    for i in range(CANT_COLUMNAS):
        if esValida(tablero, i):
            posicionesValidas.append(i)
    return posicionesValidas


In [29]:

def puntajePosicion(tablero, pieza):
    puntaje = 0
    piezaOponente = PIEZA_JUGADOR

    tableroDeEvaluacion = np.array([[3, 4, 5, 7, 5, 4, 3],
                                    [4, 6, 8, 10, 8, 6, 4],
                                    [5, 8, 11, 13, 11, 8, 5],
                                    [5, 8, 11, 13, 11, 8, 5],
                                    [4, 6, 8, 10, 8, 6, 4],
                                    [3, 4, 5, 7, 5, 4, 3]])
    
    puntajePieza = np.sum(tableroDeEvaluacion[tablero == pieza])
    puntajeOponente = np.sum(tableroDeEvaluacion[tablero == piezaOponente])
    
    puntaje = puntajePieza - puntajeOponente
    return puntaje

def isTerminalNode(tablero):
    return movimientoGanador(tablero, PIEZA_JUGADOR) or movimientoGanador(tablero, PIEZA_IA) or len(obtenerPosicionesValidas(tablero)) == 0

def minimax(tablero, profundidad, maximizandoJugador, alfa_beta_poda, alfa = None, beta = None):
    lugaresValidos = obtenerPosicionesValidas(tablero)
    esTerminal = isTerminalNode(tablero)
    
    if profundidad == 0 or esTerminal:
        if esTerminal:
            if movimientoGanador(tablero, PIEZA_IA):
                return (None, math.inf)
            elif movimientoGanador(tablero, PIEZA_JUGADOR):
                return (None, -math.inf)
            else:
                return (None, 0)
        else:
            return (None, puntajePosicion(tablero, PIEZA_IA))
        
    if maximizandoJugador:
        valor = -math.inf
        columna = random.choice(lugaresValidos)
        for col in lugaresValidos:
            fila = obtenerSiguienteFilaVacia(tablero, col)
            tableroTemporal = tablero.copy()
            soltarFicha(tableroTemporal, fila, col, PIEZA_IA)
            nuevoValor = minimax(tableroTemporal, profundidad-1, False, alfa_beta_poda, alfa, beta)[1]
            
            if nuevoValor > valor:
                valor = nuevoValor
                columna = col
            if alfa_beta_poda:  # Solo actualiza alfa y beta si alfa_beta_poda es True
                alfa = max(alfa, valor)
                if alfa >= beta:
                    break
        return columna, valor
    
    else:
        value = math.inf
        columna = random.choice(lugaresValidos)
        for col in lugaresValidos:
            fila = obtenerSiguienteFilaVacia(tablero, col)
            tableroTemporal = tablero.copy()
            soltarFicha(tableroTemporal, fila, col, PIEZA_JUGADOR)
            nuevoValor = minimax(tableroTemporal, profundidad-1, True, alfa_beta_poda, alfa, beta)[1]
            
            if nuevoValor < value:
                value = nuevoValor
                columna = col
            if alfa_beta_poda:  # Solo actualiza alfa y beta si alfa_beta_poda es True
                beta = min(beta, value)
                if alfa >= beta:
                    break
        return columna, value
    


In [42]:
# creamos el tablero
tablero = crearTablero()

# Inicializamos las variables del juego
juegoTerminado = False
turno = random.randint(JUGADOR, IA)
jugadorHumano = False
podaAlfaBeta = True
profundidadIA1 = 5
profundidadIA2 = 5

# Imprimimos el tablero
imprimirTablero(tablero)
print("\n")

while not juegoTerminado:
    if turno == JUGADOR:
        # Turno del jugador
        if jugadorHumano:
            print("---> Turno del Jugador <---")
            columna = int(input("Elije una columna (0-6): "))
            
            if esValida(tablero, columna):
                fila = obtenerSiguienteFilaVacia(tablero, columna)
                soltarFicha(tablero, fila, columna, PIEZA_JUGADOR)
                if movimientoGanador(tablero, PIEZA_JUGADOR):
                    print("Ganaste!")
                    juegoTerminado = True
                    imprimirTablero(tablero)
                    break
            
            imprimirTablero(tablero)
            print("\n") 
            turno += 1
            turno = turno % 2

        else:
            print("---> Turno de la IA-01 <---")
            columna, puntaje = minimax(tablero, profundidadIA1, True, False, -math.inf, math.inf)
            if esValida(tablero, columna):
                fila = obtenerSiguienteFilaVacia(tablero, columna)
                soltarFicha(tablero, fila, columna, PIEZA_JUGADOR)
                if movimientoGanador(tablero, PIEZA_JUGADOR):
                    print("Ganó la IA - 01!")
                    juegoTerminado = True
                    imprimirTablero(tablero)
                    break
                
            imprimirTablero(tablero)
            print("\n")
            turno += 1
            turno = turno % 2
                
    else:
        # Turno de la IA
        print("---> Turno de la IA <---")
        columna, puntaje = minimax(tablero, profundidadIA2, True, podaAlfaBeta, -math.inf, math.inf)
        if esValida(tablero, columna):
            fila = obtenerSiguienteFilaVacia(tablero, columna)
            soltarFicha(tablero, fila, columna, PIEZA_IA)
            if movimientoGanador(tablero, PIEZA_IA):
                print("Ganó la IA - 02!")
                juegoTerminado = True
                imprimirTablero(tablero)
                break
            
        imprimirTablero(tablero)
        print("\n")
        turno += 1
        turno = turno % 2
            
    
    if isTerminalNode(tablero):
        print("Empate!")
        print("\n")
        imprimirTablero(tablero)
        juegoTerminado = True

| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 

---> Turno de la IA-01 <---
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
----------------------------
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 

---> Turno de la IA <---
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 2 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
----------------------------
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 

---> Turno de la IA-01 <---
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 2 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
----------------